# Counterfactual simulation
This notebook demonstrates how to do a counterfactual simulation across different interventions and detection times in pathosim.

In [1]:
import unittest 
import pathosim as inf
import sciris as sc
import os
import numpy as np
    
result_keys = [ 
    'n_infectious',  
    'n_symptomatic', 
    'n_severe',       
    'n_recovered',   
    'n_dead']

PathoSim 3.1.2 (2022-01-16) — © 2023 by McGill University


##### Baseline simulation parameters

In [2]:
sim_pars = dict(
        use_waning    = True,           
        pop_size      = 5000,       
        pop_type      = 'behaviour_module',      
        n_days        = 100,            
        verbose       = 0,             
        rand_seed     = 42,                        
    )  

##### Intervention packages
We define two different intervention packages which each consist of several interventions

In [3]:

# interventions
school_closure = inf.change_beta(days=[40], changes=[0], layers='s')
home_office = inf.change_beta(days=[20], changes=[0.2], layers='w')
event_cancellation = inf.change_beta(days=[20], changes=[0.8], layers='c')
event_and_business_closure = inf.change_beta(days=[40], changes=[0.3], layers='c') # currently, we cannot combine several change_betas at the same layer, so they have to be combined a priori

# intervention packages (stored as dict)
packages = {
    "medium" : [home_office, event_cancellation],
    "strong" : [home_office, event_and_business_closure, school_closure]
}

##### Pathogen

In [4]:
pathogen = inf.SARS_COV_2(10)

##### Set up counterfactual simulation

In [5]:
cf = inf.CounterfactualSim(sim_pars, pathogens = [pathogen], intervention_packages=packages)

Run the baseline scenario (no interventions). This also checks whether a larger epidemic occurs in the simulated scenario and determines a realistic range of detection times.

In [6]:
cf.run_baseline()

In [7]:
print(cf.sim_baseline)
print(f"Detection range: {cf.sim_baseline.get_detection_ranges()}")
print(f"Is epidemic: {cf.sim_baseline.is_epidemic()}")

Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 9284⚙, 41☠)
Detection range: [{'lower': 8.0, 'upper': 23}]
Is epidemic: [True]


##### Run counterfactual simulations
To run a counterfactual simulation, you have to specify which intervention package should be used and what detection time to assume.

In [8]:
cf.run_counterfactual(intervention_package_key="medium", detection_times=[2, 5], verbose = True)

Running counterfactual for intervention package "medium" with detection time 2.
Running counterfactual for intervention package "medium" with detection time 5.


You can inspect the stored counterfactual simulations in `cf.sims_counterfactual`.

In [9]:
cf.sims_counterfactual

#0: 'medium': {2: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 8702⚙, 28☠), 5: Sim(<no label>; 2020-03-01 to 2020-06-09;
pop: 5000 behaviour_module; epi: 8702⚙, 28☠)}
#1: 'strong': {}


This is a two-level dictionary. The first key represents the intervention package, and the second key the detection time.

In [10]:
cf.sims_counterfactual["medium"][2]

Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠)

##### Scan detection times with counterfactual simulation

In [11]:
cf.scan_detection_range(intervention_package_keys="medium", n_steps = 3, verbose = True)

Running counterfactual for intervention package "medium" with detection time 8.
Running counterfactual for intervention package "medium" with detection time 16.
Running counterfactual for intervention package "medium" with detection time 23.


In [12]:
cf.sims_counterfactual

#0: 'medium': {2: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 8702⚙, 28☠), 5: Sim(<no label>; 2020-03-01 to 2020-06-09;
pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 8: Sim(<no label>; 2020-03-01 to
2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 16: Sim(<no label>;
2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 23:
Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi:
8702⚙, 28☠)}
#1: 'strong': {}


You can also leave out `intervention_package_keys` (or set it to `None`), in which case the detection time scan will be run for all packages.

In [13]:
cf.scan_detection_range(n_steps = 3, verbose = True)

Running counterfactual for intervention package "medium" with detection time 8.
Running counterfactual for intervention package "medium" with detection time 16.
Running counterfactual for intervention package "medium" with detection time 23.
Running counterfactual for intervention package "strong" with detection time 8.
Running counterfactual for intervention package "strong" with detection time 16.
Running counterfactual for intervention package "strong" with detection time 23.


In [14]:
cf.sims_counterfactual

#0: 'medium': {2: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 8702⚙, 28☠), 5: Sim(<no label>; 2020-03-01 to 2020-06-09;
pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 8: Sim(<no label>; 2020-03-01 to
2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 16: Sim(<no label>;
2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 23:
Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi:
8702⚙, 28☠)}
#1: 'strong': {8: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 7908⚙, 32☠), 16: Sim(<no label>; 2020-03-01 to
2020-06-09; pop: 5000 behaviour_module; epi: 7908⚙, 32☠), 23: Sim(<no label>;
2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 7908⚙, 32☠)}
